In [ ]:
import pandas as pd
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

data = pd.read_csv('my_billboard.csv', engine='python', error_bad_lines=False)
data = data[data['spotify_link'].notnull()]
data = data[data['lyrics'].notnull()]
type(data['artist'].unique())
artists = list(data['artist'].unique())[:500]
my_artists = []
for art in artists:
  if '&' not in art and ',' not in art and '+':
    my_artists.append(art)

artists = my_artists[:100]
artists.remove('Chris Stapleton')
artists.remove('Alice Merton')
artists.remove('Kenny Chesney')
artists.remove('Cole Swindell')

artists.append('Rihanna')
artists.append('Pitbull')
artists.append('Katy Perry')
artists.append('Drake')
print(artists)

data = data[data['artist'].isin(artists)]

['Juice WRLD', 'Kanye West', 'Selena Gomez', 'Cardi B', 'Luke Combs', 'Post Malone', 'Ella Mai', 'The Weeknd', 'Nicki Minaj', 'KIDS SEE GHOSTS', 'Taylor Swift', 'Daddy Yankee', 'Lil Pump', 'BTS', 'Drake', 'Kane Brown', 'Lauv', 'Blake Shelton', 'Jake Owen', 'Dua Lipa', 'Shawn Mendes', 'Famous Dex', 'J. Cole', 'Brett Young', 'Bazzi', 'Camila Cabello', 'Ariana Grande', 'YoungBoy Never Broke Again', 'Ed Sheeran', 'Rich The Kid', 'XXXTENTACION', 'Florida Georgia Line', 'Becky G + Natti Natasha', 'Jordan Davis', 'Foster The People', 'Dan + Shay', 'Childish Gambino', 'Maroon 5', 'Imagine Dragons', 'Dierks Bentley', 'Nicky Jam x J Balvin', 'Jason Aldean', '5 Seconds Of Summer', 'Bad Wolves', 'Lil Skies', 'Darius Rucker', 'Panic! At The Disco', 'Pusha T', 'Luke Bryan', 'Kevin Gates', 'Backstreet Boys', 'Sam Hunt', 'Kris Wu', 'Meghan Trainor', 'Pharrell Williams x Camila Cabello', 'Lil Baby', 'Carrie Underwood', 'Migos', 'Tank', 'Mason Ramsey', 'Portugal. The Man', 'NF', 'Brett Eldredge', 'Avici

In [ ]:
import numpy as np
n_docs = data.shape[0]

In [ ]:
print(n_docs)

192


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
sw = stopwords.words('english')

def termify():
  all_words = {}

  for row in data.iterrows():
      row = pd.Series.to_dict(row[1])
      for word in row['lyrics'].split():
          if '\n' not in word:
            if word not in all_words:
                all_words[word] = 1
            else:
                all_words[word] +=1

  keys = all_words.keys()
  n_words = len(keys)

  key_list = list(all_words.keys())
  key_list.sort()

  plots = []
  for row in data.iterrows():
      plot = pd.Series.to_dict(row[1])['lyrics']
      plots.append(plot)

  tfidf_vec = TfidfVectorizer(max_df = 0.8, min_df = 10, max_features = 5000, stop_words = 'english', norm = 'l2')
  doc_by_vocab = tfidf_vec.fit_transform(plots).toarray()
  index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}

  tf_idf = np.transpose(doc_by_vocab)
  av_tfidf = {}
  for i in range(len(index_to_vocab)):
      av_tfidf[index_to_vocab[i]] = np.sum(tf_idf[i])/n_docs

  l=list(av_tfidf.items())
  new = sorted(l, key=lambda x: x[1], reverse=True)
  #print(new)
  terms = [tup[0] for tup in new]
  top_terms = [x for x in terms if x not in sw][:300]
  return top_terms

my_terms = termify()

In [ ]:
print(my_terms)

['yeah', 'like', 'know', 'love', 'got', 'baby', 'error', 'lyrics', 'oh', 'let', 'nigga', 'ooh', 'way', 'want', 'niggas', 'make', 'time', 'bitch', 'shit', 'fuck', 'cause', 'say', 'girl', 'uh', 'feel', 'man', 'gang', 'wanna', 'em', 'right', 'heart', 'life', 'gon', 'tell', 'boy', 'come', 'need', 'mind', 'think', 'said', 'money', 'better', 'away', 'gonna', 'look', 'head', 'gotta', 'new', 'bad', 'crazy', 'ayy', 'yo', 'everybody', 'night', 'really', 'good', 'ya', 'believe', 'whip', 'alright', 'woo', 'huh', 'stay', 'lost', 'god', 'things', 'long', 'tryna', 'try', 'tonight', 'hard', 'pull', 'break', 'face', 'hey', 'pain', 'world', 'gave', 'die', 'turn', 'bout', 'stop', 'wait', 'hit', 'body', 'damn', 'lot', 'eyes', 'guess', 'song', 'care', 'high', 'black', 'days', 'movin', 'little', 'somebody', 'leave', 'lose', 'play', 'went', 'sorry', 'babe', 'hear', 'lord', 'best', 'talk', 'bitches', 'people', 'big', 'going', 'day', 'doin', 'walk', 'hold', 'count', 'young', 'thing', 'took', 'cut', 'run', 'che

In [ ]:
def docufy_csv_to_features(top_terms):
  all_docs = []
  all_X = []
  all_y = []

  for row in data.iterrows():
      row = pd.Series.to_dict(row[1])
      lyr = row['lyrics'].split()
      document_words = set(lyr)
      features = {}
      count = 0
      for word in top_terms:
          features[word] = (word in document_words)
            
      all_docs.append((features, row['artist']))
      all_X.append(features)
      all_y.append(row['artist'])

  return (all_docs, all_X, all_y)

test_all_docs = docufy_csv_to_features(my_terms)


In [ ]:
print(test_all_docs[0][1])

({'yeah': False, 'like': True, 'know': False, 'love': True, 'got': True, 'baby': False, 'error': False, 'lyrics': False, 'oh': False, 'let': True, 'nigga': False, 'ooh': False, 'way': False, 'want': True, 'niggas': False, 'make': True, 'time': True, 'bitch': True, 'shit': True, 'fuck': True, 'cause': False, 'say': True, 'girl': False, 'uh': False, 'feel': False, 'man': False, 'gang': False, 'wanna': False, 'em': False, 'right': False, 'heart': False, 'life': False, 'gon': False, 'tell': False, 'boy': False, 'come': False, 'need': False, 'mind': True, 'think': True, 'said': False, 'money': False, 'better': False, 'away': False, 'gonna': True, 'look': False, 'head': False, 'gotta': True, 'new': False, 'bad': False, 'crazy': True, 'ayy': False, 'yo': False, 'everybody': False, 'night': False, 'really': False, 'good': False, 'ya': False, 'believe': False, 'whip': False, 'alright': False, 'woo': False, 'huh': False, 'stay': False, 'lost': False, 'god': False, 'things': True, 'long': False, 

In [ ]:
test_ratio = 0.1
cutoff = int(len(test_all_docs[0]) * test_ratio)
train_set, test_set = test_all_docs[0][cutoff:], test_all_docs[0][:cutoff]
classifier1 = nltk.NaiveBayesClassifier.train(train_set)
classifier2 = SklearnClassifier(BernoulliNB()).train(train_set)
print('Achieved {0:.2f}% accuracy against training set'.format(nltk.classify.accuracy(classifier1, train_set)*100))
print('Achieved {0:.2f}% accuracy against test set'.format(nltk.classify.accuracy(classifier1, test_set)*100))

Achieved 86.94% accuracy against training set
Achieved 72.39% accuracy against test set


In [ ]:
xtest = test_all_docs[1][:cutoff]
predicted = []
for entry in xtest:
  predicted.append(classifier2.classify(entry))

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

actual = test_all_docs[2][:cutoff]
results = confusion_matrix(actual, predicted) 
  
print('Confusion Matrix :')
print(results) 
print('Report : ')
print(classification_report(actual, predicted))

Confusion Matrix :
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 3 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]
Report : 
                 precision    recall  f1-score   support

            BTS       0.00      0.00      0.00         1
        Cardi B       0.00      0.00      0.00         1
   Daddy Yankee       0.00      0.00      0.00         1
          Drake       0.00      0.00      0.00         1
       Ella Mai       0.00      0.00      0.00         

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
classifier1.show_most_informative_features(25)

Most Informative Features
                   gotta = True            Drake : Post M =      8.2 : 1.0
                    head = True               NF : Post M =      7.8 : 1.0
                    talk = True               NF : Post M =      7.8 : 1.0
                     run = True           Lil Ba : Post M =      7.8 : 1.0
                    long = True           KIDS S : Post M =      7.8 : 1.0
                      oh = True           KIDS S : Post M =      7.8 : 1.0
                    mind = True           Shawn  : Post M =      7.8 : 1.0
                  things = True               NF : Post M =      7.8 : 1.0
                   wrong = True               NF : Post M =      7.8 : 1.0
                   heart = True           Charli : Post M =      7.8 : 1.0
                    game = True           Kevin  : Post M =      7.8 : 1.0
                 friends = True           Luke B : Post M =      7.8 : 1.0
                    came = True           Russel : Post M =      7.0 : 1.0

In [ ]:
#ADD UR OWN WORDS

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 
X_train = test_all_docs[1][cutoff:]
y_train = test_all_docs[2][cutoff:]
X_test = test_all_docs[1][:cutoff]
y_test = test_all_docs[2][:cutoff]
knn = KNeighborsClassifier(n_neighbors = 7).fit([X_train], [y_train]) 
  
# accuracy on X_test 
accuracy = knn.score([X_test], [y_test]) 
print(accuracy)

TypeError: ignored